In [1]:
import pandas as pd
import numpy as np
import os
combined_df = pd.read_pickle('../scripts/data/combined_df.pickle')

In [2]:
combined_df

,Spot,Helsinki Temperature,Jyvaskyla Temperature,Rovaniemi Temperature,Consumption,Nuclear Power Production FI,Wind Power Production FI,FI-EE,FI-NO,FI-RU,FI-SE1,FI-SE3,Hydro Reservoir NO,Hydro Reservoir SE,Hydro Reservoir FI,Coal,Oil
2013-01-01 00:00:00,27.51,1.2,0.6,-0.6,9494.0,2773.600,47.6500,365.700,0.300,-1361.931,-634.040,-821.420,54099.000000,20521.000000,3793.000000,44.350000,112.300000
2013-01-01 01:00:00,24.44,1.1,0.6,-0.7,9265.0,2772.800,49.9000,312.100,0.200,-1362.781,-447.700,-821.460,54099.000000,20521.000000,3793.000000,44.350000,112.300000
2013-01-01 02:00:00,21.81,1.4,0.6,-0.9,9161.0,2773.800,51.3000,291.200,0.300,-1362.837,-368.840,-821.420,54099.000000,20521.000000,3793.000000,44.350000,112.300000
2013-01-01 03:00:00,22.37,1.9,0.4,-1.1,9240.0,2773.550,53.5000,304.600,0.300,-1388.315,-412.460,-812.600,54099.000000,20521.000000,3793.000000,44.350000,112.300000
2013-01-01 04:00:00,25.51,2.5,0.3,-1.2,9419.0,2773.750,53.7500,327.400,0.200,-1384.397,-513.030,-821.790,54099.000000,20521.000000,3793.000000,44.350000,112.300000
2013-01-01 05:00:00,27.93,2.8,0.5,-1.4,9400.0,2773.400,52.2000,362.400,0.300,-1374.759,-340.420,-821.580,54099.000000,20521.000000,3793.000000,44.350000,112.300000
2013-01-01 06:00:00,28.01,2.8,0.4,-1.3,9554.0,2773.150,52.6500,365.800,0.300,-1402.746,-270.780,-821.900,54099.000000,20521.000000,3793.000000,44.350000,112.300000
2013-01-01 07:00:00,27.80,2.9,0.4,-1.4,9524.0,2773.550,55.1500,365.700,0.200,-1387.554,-211.580,-821.630,54099.000000,20521.000000,3793.000000,44.350000,112.300000
2013-01-01 08:00:00,27.35,2.9,0.2,-1.2,9593.0,2773.350,53.5000,365.800,0.300,-1384.241,-226.000,-821.740,54099.000000,20521.000000,3793.000000,44.350000,112.300000
2013-01-01 09:00:00,28.22,2.9,0.1,-0.9,9778.0,2773.550,56.5500,365.800,0.300,-1402.657,-241.210,-821.720,54099.000000,20521.000000,3793.000000,44.350000,112.300000


In [3]:
stocks_path = 'data/stocks/'
stocks = os.listdir(stocks_path)
for stock in stocks:
    stock_df = pd.read_csv(stocks_path+stock)
    dr = pd.date_range(start='01-01-2013',end='01-01-2019',freq='D')
    stock_df['Date'] = pd.to_datetime(stock_df['Date'])
    stock_df.set_index('Date',inplace=True)
    stock_df = stock_df['01-01-2013':'31-12-2018']
    stock_df = stock_df.reindex(dr).fillna('NaN').rename_axis('Date').reset_index()
    stock_df.set_index('Date',inplace=True)
    stock_df = stock_df.astype('float64')
    stock_df = stock_df.resample('H').interpolate(method='time').bfill()[:-2]
    col_name = stock.split('.')[0]
    combined_df[f"{col_name} Close"] = stock_df['Close']

In [4]:
fingrid_path = 'data/fingrid_datasets/'
fingrid_datasets = os.listdir(fingrid_path)
for dataset in fingrid_datasets:
    fingrid_df = pd.read_csv(fingrid_path+dataset)
    fingrid_df = fingrid_df[2:]
    fingrid_df['Date'] = pd.to_datetime(fingrid_df[fingrid_df.columns[0]])
    fingrid_df.set_index('Date',inplace=True)
    fingrid_df.drop(columns=fingrid_df.columns[0],inplace=True)
    fingrid_df.drop(columns=fingrid_df.columns[:-1],inplace=True)
    fingrid_df = fingrid_df['01-01-2013':'31-12-2018'][:-1]
    print(fingrid_df.describe())
    col_name = fingrid_df.columns[0]
    combined_df[col_name] = fingrid_df[col_name]
# not useful features
combined_df.drop(columns=['Planned transmission capacity RUS-FI','Cross-border transmission fee, import from Russia'],inplace=True)

       The price of comsumption imbalance electricity
count                                    52583.000000
mean                                        36.905892
std                                         38.046484
min                                      -1000.000000
25%                                         25.480000
50%                                         32.500000
75%                                         42.440000
max                                       3000.000000
       Tuotantotasesähkön ostohinta
count                  52583.000000
mean                      32.405915
std                       17.829863
min                    -1000.000000
25%                       24.670000
50%                       31.000000
75%                       39.470000
max                      249.970000
       Sähkön tuotanto Suomessa
count              52909.000000
mean                7437.877979
std                 1403.259429
min                 4185.000000
25%                 6343.00000

In [5]:
#combined_df.drop(columns=combined_df.columns[27:76],inplace=True)
combined_df.columns
#combined_df.to_pickle('combined_df_engineered.pickle')

Index(['Spot', 'Helsinki Temperature', 'Jyvaskyla Temperature',
       'Rovaniemi Temperature', 'Consumption', 'Nuclear Power Production FI',
       'Wind Power Production FI', 'FI-EE', 'FI-NO', 'FI-RU', 'FI-SE1',
       'FI-SE3', 'Hydro Reservoir NO', 'Hydro Reservoir SE',
       'Hydro Reservoir FI', 'Coal', 'Oil', '^OMXH25 Close', 'FORTUM Close',
       '^OMXSPI Close', '^N100 Close',
       'The price of comsumption imbalance electricity',
       'Tuotantotasesähkön ostohinta', 'Sähkön tuotanto Suomessa',
       ' Tuotantotasesähkön myyntihinta'],
      dtype='object')

Converting minutes, hours etc to cyclical format. 
https://ianlondon.github.io/blog/encoding-cyclical-features-24hour-time/

In [6]:
# 2016 karkausvuosi
cyclical_time_df = pd.DataFrame(index=pd.date_range(start='01-01-2013',end='01-01-2019',freq='H'))[:-1]
cyclical_time_df['hour_sin_time'] = np.sin(2*np.pi*cyclical_time_df.index.hour/24)
cyclical_time_df['hour_cos_time'] = np.cos(2*np.pi*cyclical_time_df.index.hour/24)
# test if it was done right
#cyclical_time_df['2018-12-27'].sample(24).plot.scatter('hour_sin_time','hour_cos_time').set_aspect('equal')

cyclical_time_df['weekday_sin_time'] = np.sin(2*np.pi*cyclical_time_df.index.weekday/7)
cyclical_time_df['weekday_cos_time'] = np.cos(2*np.pi*cyclical_time_df.index.weekday/7)
# test if it was done right
#cyclical_time_df['2018-12-18':'2018-12-24'].plot.scatter('weekday_sin_time','weekday_cos_time').set_aspect('equal')

cyclical_time_df['month_sin_time'] = np.sin(2*np.pi*cyclical_time_df.index.month/12)
cyclical_time_df['month_cos_time'] = np.cos(2*np.pi*cyclical_time_df.index.month/12)
# test if it was done right
#cyclical_time_df['2013'].plot.scatter('month_sin_time','month_cos_time').set_aspect('equal')

#cyclical_time_df['day_sin_time'] = np.zeros(len(cyclical_time_df))
#cyclical_time_df['day_cos_time'] = np.zeros(len(cyclical_time_df))
temp_dfs = []
for year in ['2013','2014','2015','2016','2017','2018']:
    temp_df = cyclical_time_df[year].drop(columns=cyclical_time_df.columns)
    if year != '2016':
        temp_df['day_sin_time'] = np.sin(2*np.pi*temp_df.index.dayofyear/365)
        temp_df['day_cos_time'] = np.cos(2*np.pi*temp_df.index.dayofyear/365)
        temp_dfs.append(temp_df)
    else:
        # leap year
        temp_df['day_sin_time'] = np.sin(2*np.pi*temp_df.index.dayofyear/366)
        temp_df['day_cos_time'] = np.cos(2*np.pi*temp_df.index.dayofyear/366)
        temp_dfs.append(temp_df)
# test if it was done right
#cyclical_time_df['2016'].plot.scatter('day_sin_time','day_cos_time').set_aspect('equal')
#cyclical_time_df.describe()
cyclical_time_df[['day_sin_time','day_cos_time']] = pd.concat(temp_dfs,axis=0)
combined_df = pd.concat([combined_df,cyclical_time_df[:-1]],axis=1)
combined_df

,Spot,Helsinki Temperature,Jyvaskyla Temperature,Rovaniemi Temperature,Consumption,Nuclear Power Production FI,Wind Power Production FI,FI-EE,FI-NO,FI-RU,...,Sähkön tuotanto Suomessa,Tuotantotasesähkön myyntihinta,hour_sin_time,hour_cos_time,weekday_sin_time,weekday_cos_time,month_sin_time,month_cos_time,day_sin_time,day_cos_time
2013-01-01 00:00:00,27.51,1.2,0.6,-0.6,9494.0,2773.600,47.6500,365.700,0.300,-1361.931,...,7084.0,27.51,0.000000e+00,1.000000e+00,0.781831,0.623490,5.000000e-01,0.866025,1.721336e-02,0.999852
2013-01-01 01:00:00,24.44,1.1,0.6,-0.7,9265.0,2772.800,49.9000,312.100,0.200,-1362.781,...,6983.0,24.44,2.588190e-01,9.659258e-01,0.781831,0.623490,5.000000e-01,0.866025,1.721336e-02,0.999852
2013-01-01 02:00:00,21.81,1.4,0.6,-0.9,9161.0,2773.800,51.3000,291.200,0.300,-1362.837,...,6940.0,21.81,5.000000e-01,8.660254e-01,0.781831,0.623490,5.000000e-01,0.866025,1.721336e-02,0.999852
2013-01-01 03:00:00,22.37,1.9,0.4,-1.1,9240.0,2773.550,53.5000,304.600,0.300,-1388.315,...,6966.0,22.37,7.071068e-01,7.071068e-01,0.781831,0.623490,5.000000e-01,0.866025,1.721336e-02,0.999852
2013-01-01 04:00:00,25.51,2.5,0.3,-1.2,9419.0,2773.750,53.7500,327.400,0.200,-1384.397,...,7075.0,25.51,8.660254e-01,5.000000e-01,0.781831,0.623490,5.000000e-01,0.866025,1.721336e-02,0.999852
2013-01-01 05:00:00,27.93,2.8,0.5,-1.4,9400.0,2773.400,52.2000,362.400,0.300,-1374.759,...,7270.0,27.93,9.659258e-01,2.588190e-01,0.781831,0.623490,5.000000e-01,0.866025,1.721336e-02,0.999852
2013-01-01 06:00:00,28.01,2.8,0.4,-1.3,9554.0,2773.150,52.6500,365.800,0.300,-1402.746,...,7465.0,28.01,1.000000e+00,6.123234e-17,0.781831,0.623490,5.000000e-01,0.866025,1.721336e-02,0.999852
2013-01-01 07:00:00,27.80,2.9,0.4,-1.4,9524.0,2773.550,55.1500,365.700,0.200,-1387.554,...,7507.0,27.80,9.659258e-01,-2.588190e-01,0.781831,0.623490,5.000000e-01,0.866025,1.721336e-02,0.999852
2013-01-01 08:00:00,27.35,2.9,0.2,-1.2,9593.0,2773.350,53.5000,365.800,0.300,-1384.241,...,7575.0,27.35,8.660254e-01,-5.000000e-01,0.781831,0.623490,5.000000e-01,0.866025,1.721336e-02,0.999852
2013-01-01 09:00:00,28.22,2.9,0.1,-0.9,9778.0,2773.550,56.5500,365.800,0.300,-1402.657,...,7728.0,28.22,7.071068e-01,-7.071068e-01,0.781831,0.623490,5.000000e-01,0.866025,1.721336e-02,0.999852


Check if datetime in index is a holiday, add 1 if it is and 0 if not.

In [7]:
import holidays
fi_holidays = holidays.CountryHoliday('FI')
for day in combined_df.index:
    if day in fi_holidays:
        combined_df.loc[day,'Holiday'] = 1
    else:
        combined_df.loc[day,'Holiday'] = 0
combined_df['Holiday'] = combined_df['Holiday'].astype('int')

Use FBProphet to create new features

In [8]:
import fbprophet
spot = pd.DataFrame(columns=['ds','y'])
spot['y'] = combined_df['Spot']
spot['ds'] = combined_df.index
spot.index = pd.RangeIndex(len(spot.index))
prophet = fbprophet.Prophet(changepoint_prior_scale=0.001)
prophet.fit(spot)
forecast = prophet.make_future_dataframe(periods=0,freq='H')
forecast = prophet.predict(forecast)
forecast.set_index(['ds'],inplace=True)
combined_df = pd.concat([combined_df,forecast],axis=1)
combined_df.drop(columns=['multiplicative_terms','multiplicative_terms_lower','multiplicative_terms_upper'],inplace=True)

/home/joni/.local/lib/python3.6/site-packages/fbprophet/forecaster.py:880: FutureWarning: Series.nonzero() is deprecated and will be removed in a future version.Use Series.to_numpy().nonzero() instead
  min_dt = dt.iloc[dt.nonzero()[0]].min()


Get the hour, day and month from Datetime index
Other features
* Total Exchange = Sum of all exchanges
* Finland Temperature = Average of HEL, JKL and ROV temperatures
* Spot MA = One week rolling mean (moving average) of spot price

In [9]:
combined_df['Total Exchange'] = combined_df['FI-EE'] + combined_df['FI-NO'] + combined_df['FI-RU'] + combined_df['FI-SE1'] + combined_df['FI-SE3']
combined_df['Finland Temperature'] = combined_df[['Helsinki Temperature','Jyvaskyla Temperature','Rovaniemi Temperature']].mean(axis=1)
for i in range(1,31):
    combined_df[f"Spot T-{24*i}"] = combined_df['Spot'].shift(i*24)
    combined_df[f"Spot MA T-{24*i}"] = combined_df['Spot'].rolling(window=24*i,center=False).mean()
    combined_df[f"Spot STD T-{24*i}"] = combined_df['Spot'].rolling(window=24*i,center=False).std()
    combined_df[f"Spot Rolling Min T-{24*i}"] = combined_df['Spot'].rolling(window=24*i,center=False).min()
    combined_df[f"Spot Rolling Max T-{24*i}"] = combined_df['Spot'].rolling(window=24*i,center=False).max()

combined_df.dropna(inplace=True)
"""
combined_df['Weekday'] = combined_df.index.weekday
combined_df['Hour'] = combined_df.index.hour
combined_df['Day'] = combined_df.index.day
combined_df['Month'] = combined_df.index.month
combined_df['Quarter'] = combined_df.index.quarter"""
combined_df.columns

Index(['Spot', 'Helsinki Temperature', 'Jyvaskyla Temperature',
       'Rovaniemi Temperature', 'Consumption', 'Nuclear Power Production FI',
       'Wind Power Production FI', 'FI-EE', 'FI-NO', 'FI-RU',
       ...
       'Spot T-696', 'Spot MA T-696', 'Spot STD T-696',
       'Spot Rolling Min T-696', 'Spot Rolling Max T-696', 'Spot T-720',
       'Spot MA T-720', 'Spot STD T-720', 'Spot Rolling Min T-720',
       'Spot Rolling Max T-720'],
      dtype='object', length=204)

In [10]:
combined_df.to_pickle("combined_df_engineered_T-24.pickle")

In [34]:
with pd.option_context('display.max_rows', None, 'display.max_columns', None):
    print(combined_df.describe())

               Spot  Helsinki Temperature  Jyvaskyla Temperature  \
count  51668.000000          51668.000000           51668.000000   
mean      36.485919              6.752489               4.821106   
std       14.027401              9.090691               9.646179   
min        0.320000            -27.800000             -31.800000   
25%       28.120000              0.400000              -1.100000   
50%       34.580000              6.000000               4.000000   
75%       43.220000             13.900000              12.100000   
max      255.020000             31.200000              31.400000   

       Rovaniemi Temperature   Consumption  Nuclear Power Production FI  \
count           51668.000000  51668.000000                 51668.000000   
mean                2.494409   9442.297128                  2533.429071   
std                10.159452   1502.830446                   346.354515   
min               -31.000000   5020.000000                   824.923500   
25%         

In [33]:
columns_to_drop = [x for x in combined_df.columns if 'Spot' and 'T-' in x and int(x.split('-')[1]) < 24*2]
columns_to_drop
#with pd.option_context('display.max_rows', None, 'display.max_columns', None):
    #print(combined_df.drop(columns=columns_to_drop).describe())

['Spot T-24',
 'Spot MA T-24',
 'Spot STD T-24',
 'Spot Rolling Min T-24',
 'Spot Rolling Max T-24']

In [35]:
[combined_df.columns.get_loc(c) fo

NameError: name 'cols' is not defined